In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import OneHotEncoder
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.types import IntegerType

import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Change the number of cores in this code block
# by setting `spark.master` to `local[n]` where
# n is the number of cores

conf = pyspark.SparkConf().setAll([('spark.master', 'local[4]'),
                                   ('spark.app.name', 'Basic Setup')])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-05-18 02:51:52,660 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load Data

Read data from the `ratings.csv` file

In [3]:
movies_df = spark.read.option("header",True).csv("file:///home/work/data/movies.csv")
movies_df.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [4]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

## Add Year column

In [5]:
#Extracting the year from Movie Title
movies_df = movies_df.withColumn('movieYear',regexp_extract(col('title'), '(.+)(\()([0-9]{4})(\))', 3))

In [6]:
movies_df.show()

+-------+--------------------+--------------------+---------+
|movieId|               title|              genres|movieYear|
+-------+--------------------+--------------------+---------+
|      1|    Toy Story (1995)|Adventure|Animati...|     1995|
|      2|      Jumanji (1995)|Adventure|Childre...|     1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|     1995|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|     1995|
|      5|Father of the Bri...|              Comedy|     1995|
|      6|         Heat (1995)|Action|Crime|Thri...|     1995|
|      7|      Sabrina (1995)|      Comedy|Romance|     1995|
|      8| Tom and Huck (1995)|  Adventure|Children|     1995|
|      9| Sudden Death (1995)|              Action|     1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|     1995|
|     11|American Presiden...|Comedy|Drama|Romance|     1995|
|     12|Dracula: Dead and...|       Comedy|Horror|     1995|
|     13|        Balto (1995)|Adventure|Animati...|     1995|
|     14

## Removing IMAX
Upon visual inspection, there were only two cases for IMAX
### Case 1: IMAX was only genre
* 1 movie where this was true
    * 4460,Encounter in the Third Dimension (1999),IMAX -> only movie with IMAX as only genre -> 27 ratings
Solution: Replace pattern: ",IMAX" -> ",(no genres listed)"
### Case 2: IMAX was last genre listed
* All other 194 instances of IMAX, IMAX was the last genre
Solution: Replace pattern: "|IMAX" -> ""

In [7]:
#Check IMAX genres in records
movies_df.filter(col("genres").contains("IMAX")).show()

+-------+--------------------+--------------------+---------+
|movieId|               title|              genres|movieYear|
+-------+--------------------+--------------------+---------+
|     33|Wings of Courage ...|Adventure|Romance...|     1995|
|     37|Across the Sea of...|    Documentary|IMAX|     1995|
|    150|    Apollo 13 (1995)|Adventure|Drama|IMAX|     1995|
|    364|Lion King, The (1...|Adventure|Animati...|     1994|
|    595|Beauty and the Be...|Animation|Childre...|     1991|
|   1797|      Everest (1998)|    Documentary|IMAX|     1998|
|   3159|Fantasia 2000 (1999)|Animation|Childre...|     1999|
|   4382|       Wolves (1999)|    Documentary|IMAX|     1999|
|   4445|T-Rex: Back to th...|Adventure|Documen...|     1998|
|   4453|Michael Jordan to...|    Documentary|IMAX|     2000|
|   4454|         More (1998)|Animation|Drama|S...|     1998|
|   4455|Thrill Ride: The ...|Adventure|Documen...|     1997|
|   4456|Haunted Castle (2...|Animation|Horror|...|     2001|
|   4457

In [8]:
# Case 1: IMAX is only genre
movies_df = movies_df.withColumn('genres', regexp_replace(col('genres'), '\\|IMAX', '')) \
                    .withColumn('genres', regexp_replace(col('genres'), 'IMAX', '(no genres listed)'))
         

In [9]:
#check after IMAX genre removal 
movies_df.filter(col("genres").contains("IMAX")).show()

+-------+-----+------+---------+
|movieId|title|genres|movieYear|
+-------+-----+------+---------+
+-------+-----+------+---------+



In [10]:
movies_df.filter(col("genres") == '(no genres listed)').count()

5063

In [11]:
#Movies before flattening the genres
movies_df.count()

62423

In [12]:
print("Unique movie Counts",movies_df.select('movieId').distinct().count())

Unique movie Counts 62423


In [13]:
#Creating the flattened version of movies with genres
final_moveies_df = movies_df.select(col('movieId'),col('title'),col('movieYear'),explode(split("genres","\\|")).alias("genre"))

In [14]:
final_moveies_df.count()

112113

## Movies with no genres
Solution: Drop movies without genres
Future aim: write scraper to pull missing genres

In [15]:
#Removing movies with (no genres listed)
final_moveies_df = final_moveies_df.filter(col("genres") != "(no genres listed)")

In [16]:
distinct_genres = final_moveies_df.groupBy('genre').count().sort(col("count").desc())
print(f"Total distinct genres: {distinct_genres.count()}")
distinct_genres.show(25, truncate=False)

Total distinct genres: 18
+-----------+-----+
|genre      |count|
+-----------+-----+
|Drama      |25606|
|Comedy     |16870|
|Thriller   |8654 |
|Romance    |7719 |
|Action     |7348 |
|Horror     |5989 |
|Documentary|5605 |
|Crime      |5319 |
|Adventure  |4145 |
|Sci-Fi     |3595 |
|Children   |2935 |
|Animation  |2929 |
|Mystery    |2925 |
|Fantasy    |2731 |
|War        |1874 |
|Western    |1399 |
|Musical    |1054 |
|Film-Noir  |353  |
+-----------+-----+



In [17]:
final_moveies_df.count()

107050

In [18]:
final_moveies_df.show(truncate=False)

+-------+----------------------------------+---------+---------+
|movieId|title                             |movieYear|genre    |
+-------+----------------------------------+---------+---------+
|1      |Toy Story (1995)                  |1995     |Adventure|
|1      |Toy Story (1995)                  |1995     |Animation|
|1      |Toy Story (1995)                  |1995     |Children |
|1      |Toy Story (1995)                  |1995     |Comedy   |
|1      |Toy Story (1995)                  |1995     |Fantasy  |
|2      |Jumanji (1995)                    |1995     |Adventure|
|2      |Jumanji (1995)                    |1995     |Children |
|2      |Jumanji (1995)                    |1995     |Fantasy  |
|3      |Grumpier Old Men (1995)           |1995     |Comedy   |
|3      |Grumpier Old Men (1995)           |1995     |Romance  |
|4      |Waiting to Exhale (1995)          |1995     |Comedy   |
|4      |Waiting to Exhale (1995)          |1995     |Drama    |
|4      |Waiting to Exhal

## Movies with no ratings
Interestingly, the movies found with no ratings were found to be a mixed bag of quality. A decent check of movies without ratings were also older movies.
Solution: Drop movies since we have a way of guestimating their rating

Future Aims: Pull average rating from other sites and possibly introduce movies as a fresh experience for users.

# How to deal with genres?
## One hot encoding
```
Action           001
Action|Adventure 010
Drama            100
Dram|Action      110
```
https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.feature.OneHotEncoder.html#:~:text=A%20one%2Dhot%20encoder%20that,0.0%2C%201.0%2C%200.0%5D%20
## Writing CSV
Convert 18 long array to string representation

In [19]:

genre_list = np.array(final_moveies_df.select(col('genre')).distinct().collect()).squeeze().tolist()
genre_map = {g: i for i, g in enumerate(genre_list)}

# https://stackoverflow.com/questions/42980704/pyspark-create-new-column-with-mapping-from-a-dict

#This UDF converts the genre_id to mapped numerical value
def translate(mapping):
    def translate_(col):
        return mapping.get(col)
    return udf(translate_, IntegerType())

final_moveies_df = final_moveies_df.withColumn("genre_id", translate(genre_map)("genre"))

In [20]:
ohe = OneHotEncoder(inputCol='genre_id', outputCol='ohe_test', dropLast=False)

ohe_model = ohe.fit(final_moveies_df)
transformed_df = ohe_model.transform(final_moveies_df)
transformed_df.show()

+-------+--------------------+---------+---------+--------+---------------+
|movieId|               title|movieYear|    genre|genre_id|       ohe_test|
+-------+--------------------+---------+---------+--------+---------------+
|      1|    Toy Story (1995)|     1995|Adventure|       3| (18,[3],[1.0])|
|      1|    Toy Story (1995)|     1995|Animation|      10|(18,[10],[1.0])|
|      1|    Toy Story (1995)|     1995| Children|      15|(18,[15],[1.0])|
|      1|    Toy Story (1995)|     1995|   Comedy|      14|(18,[14],[1.0])|
|      1|    Toy Story (1995)|     1995|  Fantasy|       7| (18,[7],[1.0])|
|      2|      Jumanji (1995)|     1995|Adventure|       3| (18,[3],[1.0])|
|      2|      Jumanji (1995)|     1995| Children|      15|(18,[15],[1.0])|
|      2|      Jumanji (1995)|     1995|  Fantasy|       7| (18,[7],[1.0])|
|      3|Grumpier Old Men ...|     1995|   Comedy|      14|(18,[14],[1.0])|
|      3|Grumpier Old Men ...|     1995|  Romance|       1| (18,[1],[1.0])|
|      4|Wai

In [21]:
#Testing a sparse vector in array form
ohe_model.transform(final_moveies_df).collect()[1].ohe_test.toArray()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0.])

In [22]:
#Uncomment for testing
#sc = spark.sparkContext

#df = sc.parallelize([
#  (1, SparseVector(17, {3: 1.0})),
#  (2, SparseVector(17, {10: 1.0})),
#  (3, SparseVector(17, {15: 1.0})),
#  (4, SparseVector(17, {14: 1.0})),
#  (5, SparseVector(17, {7: 1.0})),
#]).toDF(["row_num", "features"])

#df.printSchema()
#df.show(truncate=False)

In [23]:
from pyspark.ml.linalg import SparseVector, DenseVector
# note that using Sparse and Dense Vectors from ml.linalg. There are other Sparse/Dense vectors in spark.

#This is user defined function to convert the sparse vector to array for easy parsing
def sparse_to_array(v):
  v = DenseVector(v)
  new_array = list([float(x) for x in v])
  return new_array

sparse_to_array_udf = F.udf(sparse_to_array, T.ArrayType(T.FloatType()))


In [24]:
final_moveies_df = transformed_df.withColumn('ohe_array', sparse_to_array_udf('ohe_test')) \
    .select("movieId","title","movieYear","genre","ohe_array") \
    .groupBy("movieId","title","movieYear") \
    .agg( 
        concat_ws('|',collect_list(col("genre"))).alias("genres"),
        F.array(*[F.sum(F.col('ohe_array')[i]) for i in range(len(genre_map))]).alias('dense_ohe_feature')
    )

In [25]:
final_moveies_df.show(5, truncate=False)

+-------+-------------------------------------+---------+-------------------------+------------------------------------------------------------------------------------------+
|movieId|title                                |movieYear|genres                   |dense_ohe_feature                                                                         |
+-------+-------------------------------------+---------+-------------------------+------------------------------------------------------------------------------------------+
|10     |GoldenEye (1995)                     |1995     |Action|Adventure|Thriller|[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]|
|1000   |Curdled (1996)                       |1996     |Crime                    |[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]|
|100003 |Up in Smoke (1957)                   |1957     |Comedy                   |[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [26]:
#Saving the file
#final_moveies_df.coalesce(1).write.csv("file:///home/work/data/movies_preped", header=True)

In [27]:
#final_moveies_df.select('movieId', 'dense_ohe_feature.*').show()
final_moveies_df = final_moveies_df.select(['movieId','title','movieYear','genres'] + [final_moveies_df.dense_ohe_feature[x] for x in range(len(genre_map))])

In [28]:
final_moveies_df.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- movieYear: string (nullable = true)
 |-- genres: string (nullable = false)
 |-- dense_ohe_feature[0]: double (nullable = true)
 |-- dense_ohe_feature[1]: double (nullable = true)
 |-- dense_ohe_feature[2]: double (nullable = true)
 |-- dense_ohe_feature[3]: double (nullable = true)
 |-- dense_ohe_feature[4]: double (nullable = true)
 |-- dense_ohe_feature[5]: double (nullable = true)
 |-- dense_ohe_feature[6]: double (nullable = true)
 |-- dense_ohe_feature[7]: double (nullable = true)
 |-- dense_ohe_feature[8]: double (nullable = true)
 |-- dense_ohe_feature[9]: double (nullable = true)
 |-- dense_ohe_feature[10]: double (nullable = true)
 |-- dense_ohe_feature[11]: double (nullable = true)
 |-- dense_ohe_feature[12]: double (nullable = true)
 |-- dense_ohe_feature[13]: double (nullable = true)
 |-- dense_ohe_feature[14]: double (nullable = true)
 |-- dense_ohe_feature[15]: double (nullable = true)
 

In [29]:
final_moveies_df.show(5, truncate=False)

+-------+-------------------------------------+---------+-------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
|movieId|title                                |movieYear|genres                   |dense_ohe_feature[0]|dense_ohe_feature[1]|dense_ohe_feature[2]|dense_ohe_feature[3]|dense_ohe_feature[4]|dense_ohe_feature[5]|dense_ohe_feature[6]|dense_ohe_feature[7]|dense_ohe_feature[8]|dense_ohe_feature[9]|dense_ohe_feature[10]|dense_ohe_feature[11]|dense_ohe_feature[12]|dense_ohe_feature[13]|dense_ohe_feature[14]|dense_ohe_feature[15]|dense_ohe_feature[16]|dense_ohe_feature[17]|
+-------+-------------------------------------+---------+---

*To get the corresponding genre key*

In [30]:
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val][0]

*Renaming all the genre columns*

In [31]:
for i in range(len(genre_map)):
    old_column = "dense_ohe_feature[{}]".format(i)
    new_column = get_keys_from_value(genre_map,i)
    final_moveies_df = final_moveies_df.withColumnRenamed(old_column,new_column)
final_moveies_df.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- movieYear: string (nullable = true)
 |-- genres: string (nullable = false)
 |-- Crime: double (nullable = true)
 |-- Romance: double (nullable = true)
 |-- Thriller: double (nullable = true)
 |-- Adventure: double (nullable = true)
 |-- Drama: double (nullable = true)
 |-- War: double (nullable = true)
 |-- Documentary: double (nullable = true)
 |-- Fantasy: double (nullable = true)
 |-- Mystery: double (nullable = true)
 |-- Musical: double (nullable = true)
 |-- Animation: double (nullable = true)
 |-- Film-Noir: double (nullable = true)
 |-- Horror: double (nullable = true)
 |-- Western: double (nullable = true)
 |-- Comedy: double (nullable = true)
 |-- Children: double (nullable = true)
 |-- Action: double (nullable = true)
 |-- Sci-Fi: double (nullable = true)



In [32]:
final_moveies_df.show(5, truncate=False)

+-------+-------------------------------------+---------+-------------------------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------+-------+------+--------+------+------+
|movieId|title                                |movieYear|genres                   |Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|Horror|Western|Comedy|Children|Action|Sci-Fi|
+-------+-------------------------------------+---------+-------------------------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------+-------+------+--------+------+------+
|10     |GoldenEye (1995)                     |1995     |Action|Adventure|Thriller|0.0  |0.0    |1.0     |1.0      |0.0  |0.0|0.0        |0.0    |0.0    |0.0    |0.0      |0.0      |0.0   |0.0    |0.0   |0.0     |1.0   |0.0   |
|1000   |Curdled (1996)                       |1996     |Crime                    |1.0  

In [33]:
final_moveies_df.count()

57360

## Add average movie ratings

In [34]:
ratings_df = spark.read.csv("file:///home/work/data/ratings.csv", inferSchema=True, header=True)
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [35]:
final_moveies_df.createOrReplaceTempView("m")
ratings_df.createOrReplaceTempView("r")

In [36]:
avg_rating_5 = spark.sql("SELECT r.movieId, AVG(r.rating) as avg_rating FROM r GROUP BY r.movieId HAVING COUNT(r.userId) >= 5 ORDER BY avg_rating DESC")
avg_rating_5.createOrReplaceTempView("avg_5")
avg_rating_10 = spark.sql("SELECT r.movieId, AVG(r.rating) as avg_rating FROM r GROUP BY r.movieId HAVING COUNT(r.userId) >= 10 ORDER BY avg_rating DESC")
avg_rating_10.createOrReplaceTempView("avg_10")
print(f"Total distinct movies with min 5:  {avg_rating_5.count()}")
print(f"Total distinct movies with min 10: {avg_rating_10.count()}")
avg_rating_10.show()

Total distinct movies with min 5:  32720


Total distinct movies with min 10: 24330


+-------+------------------+
|movieId|        avg_rating|
+-------+------------------+
| 171011| 4.483096085409253|
| 159817| 4.464796794504865|
|    318| 4.413576004516335|
| 170705| 4.398598820058997|
| 158958| 4.384615384615385|
| 171331|             4.375|
| 169022| 4.333333333333333|
| 171495|4.3267148014440435|
|    858| 4.324336165187245|
| 179135| 4.289833080424886|
|     50| 4.284353213163313|
| 174551| 4.277777777777778|
| 172631|4.2727272727272725|
| 198185| 4.267361111111111|
|   1221|4.2617585117585115|
| 163809| 4.258241758241758|
| 176601| 4.256578947368421|
|   2019|  4.25476920775043|
| 147250|4.2517006802721085|
| 131980|              4.25|
+-------+------------------+
only showing top 20 rows



In [37]:
print(f"Total distinct ratings with min 5:")
spark.sql("SELECT COUNT(r.movieId) FROM r INNER JOIN avg_5 ON r.movieId == avg_5.movieId").show()
print(f"Total distinct ratings with min 10:")
spark.sql("SELECT COUNT(r.movieId) FROM r INNER JOIN avg_10 ON r.movieId == avg_10.movieId").show()

Total distinct ratings with min 5:


+--------------+
|count(movieId)|
+--------------+
|      24945870|
+--------------+

Total distinct ratings with min 10:


+--------------+
|count(movieId)|
+--------------+
|      24890583|
+--------------+



### Filter by at least 5 ratings/movie  
* Movies drop down from 57360 to 32720 (24640 total movies lost)  
* Ratings drop from 25000095 to 24945870 (54225 total ratings lost)  
### Filter by at least 10 ratings/movie  
* Movies drop down from 57360 to 24330 (33030 total movies lost)  
* Ratings drop from 25000095 to 24890583 (109512 total ratings lost)  

In [39]:
# join average ratings with movie table
final_df_cols = ["m.`" + c + "`" if '-' in c else "m." + c for c in final_moveies_df.columns]

final_df_cols.insert(2, "avg_5.avg_rating")
final_df_cols = ", ".join(final_df_cols)

# put together query
query = "SELECT " + final_df_cols + " FROM m JOIN avg_5 ON m.movieId == avg_5.movieId"

# run query
final_moveies_df_5 = spark.sql(query)
# final_moveies_df_5.show()

In [40]:
# join average ratings with movie table
final_df_cols = ["m.`" + c + "`" if '-' in c else "m." + c for c in final_moveies_df.columns]

final_df_cols.insert(2, "avg_10.avg_rating")
final_df_cols = ", ".join(final_df_cols)

# put together query
query = "SELECT " + final_df_cols + " FROM m JOIN avg_10 ON m.movieId == avg_10.movieId"

# run query
final_moveies_df_10 = spark.sql(query)
# final_moveies_df_10.show()

In [41]:
write_files = False
if write_files:
    final_moveies_df_5.coalesce(1).write.csv("hdfs:///final_movies_df_5", header=True)
    !hadoop fs -ls /final_moveies_df_5
    !hadoop fs -copyToLocal /final_moveies_df_5/part-*.csv /home/work/data/

    final_moveies_df_10.coalesce(1).write.csv("hdfs:///final_movies_df_10", header=True)
    !hadoop fs -ls /final_moveies_df_10
    !hadoop fs -copyToLocal /final_moveies_df_10/part-*.csv /home/work/data/

Saving the final cleaned CSV file.

In [42]:
#!hadoop fs -copyToLocal /final_moveies_df/part-*.csv /home/work/data/
#!mv /home/work/data/part-00000-7c9c7e1a-ce8f-410b-878e-69c76b9a5bd5-c000.csv /home/work/data/cleaned_movies.csv
#!ls -lrt /home/work/data/